In [1]:
import pandas as pd
import numpy as np

In [2]:
df_01 = pd.read_csv("./data/highway_01_07.csv")
df_02 = pd.read_csv("./data/highway_08_12.csv")

In [3]:
def reform():
    new_list = []
    for i in range(df_02.shape[0]):
        kor = ["경기", "충청","전라","경상","강원"]
        eng = ["Gyeonggi", "Chungcheong", "Jeolla", "Gyeongsang", "Gangwon"]
        for j in range(5):
            if df_02["StartPoint"][i] == kor[j]:
                new_list.append(eng[j])
    df_02["StartPoint"] = new_list

reform()

In [4]:
df_02.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140801,Gangwon,63365.0,7044.0,892.0,1756.0,140258.0
1,20140801,Gyeonggi,553781.0,110895.0,9961.0,12596.0,89665.0


In [5]:
df_01.head(1)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,Chungcheong,83318.0,104617.0,4147.0,4126.0,6021.0


In [6]:
df = pd.concat([df_01, df_02])

In [7]:
df

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,Chungcheong,83318.0,104617.0,4147.0,4126.0,6021.0
1,20140101,Gangwon,57398.0,5783.0,427.0,991.0,102850.0
2,20140101,Gyeonggi,373199.0,82263.0,3114.0,5841.0,51386.0
3,20140101,Gyeongsang,6926.0,5368.0,15514.0,334217.0,986.0
4,20140101,Jeolla,3298.0,4759.0,130474.0,14832.0,478.0
...,...,...,...,...,...,...,...
760,20141231,Gangwon,45095.0,5978.0,676.0,1243.0,81547.0
761,20141231,Gyeonggi,558204.0,89242.0,4809.0,9903.0,59000.0
762,20141231,Gyeongsang,9604.0,5657.0,14685.0,380287.0,1538.0
763,20141231,Jeolla,3679.0,4224.0,134007.0,14128.0,640.0


In [8]:
def exec():
    eng = ["Gyeonggi", "Chungcheong", "Jeolla", "Gyeongsang", "Gangwon"]
    list = []
    target = df.loc[df["StartPoint"] == "Gangwon"].reset_index(drop=True)
    for j in range(5):
        for i in range(target.shape[0]):
            list.append(target[eng[j]][i])
        
    return list

In [9]:
rtn = exec()
gangwonTotal = pd.DataFrame(rtn, columns=["val"])
gangwonTotal.sort_values(by=["val"])

,val
861,315.0
770,327.0
771,328.0
806,329.0
765,330.0
...,...
1687,147135.0
1673,160509.0
1710,163533.0
1584,176174.0


In [18]:
q1 = int(round(gangwonTotal.quantile(0.25),0))
q2 = int(round(gangwonTotal.quantile(0.5),0))
q3 = int(round(gangwonTotal.quantile(0.75),0))
print(q1, ' / ', q2 , '/', q3)

988  /  5455 / 54882


In [48]:
def to_number(x):
    return int(x)

In [64]:
fromGyeonggi = df[df["StartPoint"] == "Gyeonggi"][["Gyeongsang"]]
fromGyeongsang = df[df["StartPoint"] == "Gyeongsang"][["Gyeonggi"]]

In [42]:
from scipy.stats import ttest_ind

In [65]:
ttest_ind(fromGyeonggi, fromGyeongsang)

Ttest_indResult(statistic=array([-3.22662439]), pvalue=array([0.00130866]))

In [88]:
def parseToDate(x):
    x = str(x)
    year = str(x[0:4])
    month = str(x[4:6])
    day = str(x[6:8])
    return year + '-' + month + '-' + day

In [128]:
fromChungcheong = df[df["StartPoint"] == "Chungcheong"][["date","Gangwon"]].reset_index(drop=True)
fromChungcheong["date"] = fromChungcheong["date"].apply(lambda x:parseToDate(x))
fromChungcheong["date"] = pd.to_datetime(fromChungcheong["date"])
fromChungcheong["month"] = fromChungcheong["date"].dt.month
fromChungcheong["day"] = fromChungcheong["date"].dt.weekday
fromChungcheong.head(2)

,date,Gangwon,month,day
0,2014-01-01,6021.0,1,2
1,2014-01-02,5270.0,1,3


In [138]:
groupResult = fromChungcheong.groupby(["day", "month"]).agg({"Gangwon":"mean"}).reset_index()
groupResult

,day,month,Gangwon
0,0,1,5094.50
1,0,2,5131.00
2,0,3,5276.40
3,0,4,5530.50
4,0,5,5940.00
...,...,...,...
79,6,8,6080.60
80,6,9,5797.50
81,6,10,5913.00
82,6,11,5612.40


In [197]:
def minMaxScaler(x):
    return (x - x.min()) / (x.max()-x.min())

result = []
result.append(minMaxScaler(groupResult[groupResult["day"] == 0]["Gangwon"]))
result.append(minMaxScaler(groupResult[groupResult["day"] == 1]["Gangwon"]))
result.append(minMaxScaler(groupResult[groupResult["day"] == 2]["Gangwon"]))
result.append(minMaxScaler(groupResult[groupResult["day"] == 3]["Gangwon"]))
result.append(minMaxScaler(groupResult[groupResult["day"] == 4]["Gangwon"]))
result.append(minMaxScaler(groupResult[groupResult["day"] == 5]["Gangwon"]))
result.append(minMaxScaler(groupResult[groupResult["day"] == 6]["Gangwon"]))
result = np.array(result).flatten()
groupResult["nor_minmax"] = result

0     0.000000
1     0.018383
2     0.091614
3     0.219592
4     0.425837
5     0.320675
6     0.585621
7     1.000000
8     0.671619
9     0.529338
10    0.722488
11    0.476303
12    0.001116
13    0.000000
14    0.042405
15    0.143088
16    0.236295
17    0.330869
18    0.458781
19    1.000000
20    0.330004
21    0.577765
22    0.652532
23    0.442823
Name: Gangwon, dtype: float64

In [202]:
groupResult.head(12)

,day,month,Gangwon,nor_minmax
0,0,1,5094.50,0.000000
1,0,2,5131.00,0.018383
2,0,3,5276.40,0.091614
3,0,4,5530.50,0.219592
4,0,5,5940.00,0.425837
5,0,6,5731.20,0.320675
6,0,7,6257.25,0.585621
7,0,8,7080.00,1.000000
8,0,9,6428.00,0.671619
9,0,10,6145.50,0.529338


In [215]:
nor_month_mean = groupResult.groupby("month")["nor_minmax"].agg("mean")
nor_month_mean = nor_month_mean.reset_index()
nor_month_mean

,month,nor_minmax
0,1,0.093441
1,2,0.012351
2,3,0.083933
3,4,0.194594
4,5,0.390898
5,6,0.410114
6,7,0.514162
7,8,0.989239
8,9,0.469974
9,10,0.722010


In [216]:
from sklearn.cluster import KMeans

In [256]:
resultKmeans = KMeans(n_clusters=3).fit(pd.DataFrame(nor_month_mean["nor_minmax"]))
resultKmeans

KMeans(n_clusters=3)

In [257]:
nor_month_mean["cluster"] = resultKmeans.labels_

In [258]:
nor_month_mean

,month,nor_minmax,cluster
0,1,0.093441,0
1,2,0.012351,0
2,3,0.083933,0
3,4,0.194594,0
4,5,0.390898,2
5,6,0.410114,2
6,7,0.514162,2
7,8,0.989239,1
8,9,0.469974,2
9,10,0.722010,1


In [266]:
targetGroup = pd.concat([
    groupResult[groupResult["month"] == 8],
    groupResult[groupResult["month"] == 10],
    groupResult[groupResult["month"] == 11]
])
targetGroup

,day,month,Gangwon,nor_minmax
7,0,8,7080.00,1.000000
19,1,8,6857.00,1.000000
31,2,8,7076.25,1.000000
43,3,8,7184.75,1.000000
55,4,8,8782.80,1.000000
67,5,8,8352.00,0.924674
79,6,8,6080.60,1.000000
9,0,10,6145.50,0.529338
21,1,10,6100.25,0.577765
33,2,10,6449.60,0.670834


In [267]:
targetGroup[targetGroup["day"] == 0].agg({"nor_minmax":"mean"})

nor_minmax    0.750609
dtype: float64

In [367]:
3 0.75

SyntaxError: invalid syntax (Temp/ipykernel_3440/920186129.py, line 1)

In [295]:
from datetime import timedelta

In [368]:
fromGyeonggi = df[df["StartPoint"] == "Gyeonggi"][["date","Chungcheong","Gyeongsang", "Gangwon","Jeolla"]].reset_index(drop=True)
fromGyeonggi["date"] = fromGyeonggi["date"].apply(lambda x:parseToDate(x))
fromGyeonggi["date"] = pd.to_datetime(fromGyeonggi["date"])
fromGyeonggi["week"] = fromGyeonggi["date"].dt.isocalendar().week
fromGyeonggi["day"] = fromGyeonggi["date"].dt.weekday
temp = pd.concat([
    fromGyeonggi[(fromGyeonggi["week"] == 27) & (fromGyeonggi["day"] == 6)],
    fromGyeonggi[(fromGyeonggi["week"] == 28) & (fromGyeonggi["day"] == 6)],
    fromGyeonggi[(fromGyeonggi["week"] == 29) & (fromGyeonggi["day"] == 6)]
]).reset_index(drop=True)
temp2 = pd.concat([
    fromGyeonggi[(fromGyeonggi["week"] == 27) & (fromGyeonggi["day"] == 5)],
    fromGyeonggi[(fromGyeonggi["week"] == 28) & (fromGyeonggi["day"] == 5)],
    fromGyeonggi[(fromGyeonggi["week"] == 29) & (fromGyeonggi["day"] == 5)]
]).reset_index(drop=True)
temp2
temp["Jeolla_before"] = temp2["Jeolla"]
temp

,date,Chungcheong,Gyeongsang,Gangwon,Jeolla,week,day,Jeolla_before
0,2014-07-06,84761.0,5460.0,44771.0,NaN,27,6,4837.0
1,2014-07-13,85702.0,5492.0,47001.0,NaN,28,6,5082.0
2,2014-07-20,82962.0,5356.0,47133.0,NaN,29,6,4434.0


In [369]:
fromGyeonggi

,date,Chungcheong,Gyeongsang,Gangwon,Jeolla,week,day
0,2014-01-01,82263.0,5841.0,51386.0,3114.0,1,2
1,2014-01-02,89221.0,6662.0,43191.0,3046.0,1,3
2,2014-01-03,91017.0,8137.0,52386.0,3719.0,1,4
3,2014-01-04,89233.0,7747.0,60970.0,4413.0,1,5
4,2014-01-05,74418.0,5703.0,40280.0,3058.0,1,6
...,...,...,...,...,...,...,...
360,2014-12-27,96811.0,9157.0,63668.0,5272.0,52,5
361,2014-12-28,81343.0,7192.0,45513.0,3423.0,52,6
362,2014-12-29,99086.0,6966.0,47373.0,3188.0,1,0
363,2014-12-30,89472.0,7647.0,45422.0,3398.0,1,1


In [370]:
trainDate = fromGyeonggi.dropna(subset=["Jeolla"]).reset_index(drop=True)
trainDate["Jeolla_before"] = trainDate["Jeolla"].shift(periods=1)

In [371]:
trainDate = trainDate.dropna().reset_index()
trainDate

,index,date,Chungcheong,Gyeongsang,Gangwon,Jeolla,week,day,Jeolla_before
0,1,2014-01-02,89221.0,6662.0,43191.0,3046.0,1,3,3114.0
1,2,2014-01-03,91017.0,8137.0,52386.0,3719.0,1,4,3046.0
2,3,2014-01-04,89233.0,7747.0,60970.0,4413.0,1,5,3719.0
3,4,2014-01-05,74418.0,5703.0,40280.0,3058.0,1,6,4413.0
4,5,2014-01-06,91474.0,5950.0,42085.0,2936.0,2,0,3058.0
...,...,...,...,...,...,...,...,...,...
356,357,2014-12-27,96811.0,9157.0,63668.0,5272.0,52,5,3922.0
357,358,2014-12-28,81343.0,7192.0,45513.0,3423.0,52,6,5272.0
358,359,2014-12-29,99086.0,6966.0,47373.0,3188.0,1,0,3423.0
359,360,2014-12-30,89472.0,7647.0,45422.0,3398.0,1,1,3188.0


In [379]:
filteredTrainData = trainDate[(trainDate["date"]<"2014-07-01") & (trainDate["day"]==6)]

,index,date,Chungcheong,Gyeongsang,Gangwon,Jeolla,week,day,Jeolla_before
150,151,2014-06-01,85961.0,5155.0,44214.0,2849.0,22,6,5137.0
157,158,2014-06-08,81795.0,7560.0,41344.0,3801.0,23,6,4470.0
164,165,2014-06-15,86842.0,5180.0,45036.0,2889.0,24,6,5014.0
171,172,2014-06-22,88471.0,5620.0,45987.0,3050.0,25,6,4788.0
178,179,2014-06-29,84856.0,5407.0,44949.0,2980.0,26,6,5058.0


In [372]:
from sklearn.linear_model import LinearRegression

In [380]:
model = LinearRegression().fit(filteredTrainData.loc[:,["Chungcheong", "Gyeongsang","Gangwon", "Jeolla_before"]],
                               filteredTrainData.loc[:,"Jeolla"])

In [381]:
model.predict(temp.loc[:,["Chungcheong", "Gyeongsang","Gangwon", "Jeolla_before"]])

array([2962.77160195, 3161.43805346, 3027.50554579])

In [375]:
[2962.8, 3161.4 , 3027.5]

[2296.6, 2380.7, 2098.1]